https://github.com/AI4Finance-Foundation

* 如果使用python3.8的话，requirements可以都装上，但是fin-rl库没法安装，因为需要3.11的python。
* 如果使用py3.11的话，requirements不能都装上，那就不装了，直接把Fin-RL库的代码下载到本地，进去，然后“pip install .”就好了。
* 然后下载FinRL-Meta库代码到本地，工作目录改到其下即可。
* 然后就缺啥库补啥库好了。

In [1]:
# 我老妈在家里研究股票。
# 挣了钱就过来跟我炫耀：
# 是不是比你的时薪还要高。
# 我每天卷得像我的裤脚。
# 自己的时间太少，每天晚上舍不得睡觉。

In [2]:
import os
os.chdir("/Users/minkexiu/Desktop/FinRL-Meta-master/")

In [3]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
from IPython import display
display.set_matplotlib_formats("svg") ## 设置一下，这样一来，图片就是svg格式的矢量图了。

In [ ]:
from meta import config
from meta.data_processor import DataProcessor
from main import check_and_make_directories
from meta.data_processors.tushare import Tushare, ReturnPlotter
from meta.env_stock_trading.env_stocktrading_China_A_shares import (
    StockTradingEnv,
)

In [2]:
from agents.stablebaselines3_models import DRLAgent
import os
from typing import List
from argparse import ArgumentParser
from meta import config
from meta.config_tickers import DOW_30_TICKER
from meta.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
    ERL_PARAMS,
    RLlib_PARAMS,
    SAC_PARAMS,
    ALPACA_API_KEY,
    ALPACA_API_SECRET,
    ALPACA_API_BASE_URL,
)

In [3]:
pd.options.display.max_columns = None

print("ALL Modules have been imported!")

ALL Modules have been imported!


In [4]:
"""
use check_and_make_directories() to replace the following

if not os.path.exists("./datasets"):
    os.makedirs("./datasets")
if not os.path.exists("./trained_models"):
    os.makedirs("./trained_models")
if not os.path.exists("./tensorboard_log"):
    os.makedirs("./tensorboard_log")
if not os.path.exists("./results"):
    os.makedirs("./results")
"""

check_and_make_directories(
    [DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR]
)

In [5]:
### Download data, cleaning and feature engineering

ticker_list = [
    "600000.SH",
    "600009.SH",
    "600016.SH",
    "600028.SH",
    "600030.SH",
    "600031.SH",
    "600036.SH",
    "600050.SH",
    "600104.SH",
    "600196.SH",
    "600276.SH",
    "600309.SH",
    "600519.SH",
    "600547.SH",
    "600570.SH",
]

TRAIN_START_DATE = "2015-01-01"
TRAIN_END_DATE = "2019-08-01"
TRADE_START_DATE = "2019-08-01"
TRADE_END_DATE = "2020-01-03"


TIME_INTERVAL = "1d"
kwargs = {}
kwargs["token"] = "53b6254438e6d307b6799dbd575883528bc1da912f3d6a31159ecc9e"
p = DataProcessor(
    data_source="tushare",
    start_date=TRAIN_START_DATE,
    end_date=TRADE_END_DATE,
    time_interval=TIME_INTERVAL,
    **kwargs,
)

tushare successfully connected


In [6]:
# download and clean
p.download_data(ticker_list=ticker_list)
p.clean_data()
p.fillna()

100%|███████████████████████████████████████████| 15/15 [00:34<00:00,  2.32s/it]

Download complete! Dataset saved to ./data/dataset.csv. 
Shape of DataFrame: (17960, 8)
Shape of DataFrame:  (18315, 8)


In [8]:
# add_technical_indicator
p.add_technical_indicator(config.INDICATORS)
p.fillna()
print(f"p.dataframe: ")
p.dataframe

tech_indicator_list:  ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']
indicator:  macd
indicator:  boll_ub
indicator:  boll_lb
indicator:  rsi_30
indicator:  cci_30
indicator:  dx_30
indicator:  close_30_sma
indicator:  close_60_sma
Succesfully add technical indicators
Shape of DataFrame:  (18270, 17)
p.dataframe: 


,tic,time,index,open,high,low,close,adjusted_close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,600000.SH,2015-01-08,45,15.87,15.88,15.20,15.25,15.25,3306271.72,-0.032571,16.617911,15.012089,6.058641,-125.593009,23.014040,15.815000,15.815000
1,600009.SH,2015-01-08,46,20.18,20.18,19.73,20.00,20.00,198117.45,-0.016008,20.663897,19.736103,12.828915,-90.842491,100.000000,20.200000,20.200000
2,600016.SH,2015-01-08,47,10.61,10.66,10.09,10.20,10.20,4851684.17,-0.018247,10.957604,9.997396,11.862558,-99.887006,100.000000,10.477500,10.477500
3,600028.SH,2015-01-08,48,7.09,7.41,6.83,6.85,6.85,8190902.35,-0.008227,7.342000,6.743000,27.409248,36.578171,64.934862,7.042500,7.042500
4,600030.SH,2015-01-08,49,36.40,36.70,34.68,35.25,35.25,6376268.69,0.032910,36.576444,33.808556,61.517448,47.947020,100.000000,35.192500,35.192500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18265,600276.SH,2020-01-03,18310,88.02,88.20,85.70,85.95,85.95,218177.79,-0.301525,87.731004,82.917996,50.964123,32.816522,9.873974,85.918667,87.503333
18266,600309.SH,2020-01-03,18311,55.90,57.76,55.90,56.94,56.94,177848.57,1.921017,56.960111,49.500889,69.086313,159.628783,51.722600,51.771333,48.518333
18267,600519.SH,2020-01-03,18312,1117.00,1117.00,1076.90,1078.56,1078.56,130318.78,-9.821047,1199.012393,1105.801607,41.484737,-219.877404,36.358177,1153.569333,1174.064000
18268,600547.SH,2020-01-03,18313,33.21,35.25,33.10,34.78,34.78,987248.95,0.543193,33.565047,30.068953,58.038366,290.350386,54.036298,31.433667,31.491667


In [15]:
### Split traning dataset

In [10]:
train = p.data_split(p.dataframe, TRAIN_START_DATE, TRAIN_END_DATE)
print(f"len(train.tic.unique()): {len(train.tic.unique())}")

len(train.tic.unique()): 15


In [11]:
train.tic.unique()

array(['600000.SH', '600009.SH', '600016.SH', '600028.SH', '600030.SH',
       '600031.SH', '600036.SH', '600050.SH', '600104.SH', '600196.SH',
       '600276.SH', '600309.SH', '600519.SH', '600547.SH', '600570.SH'],
      dtype=object)

In [12]:
train.head()

,tic,time,index,open,high,low,close,adjusted_close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,600000.SH,2015-01-08,45,15.87,15.88,15.20,15.25,15.25,3306271.72,-0.032571,16.617911,15.012089,6.058641,-125.593009,23.014040,15.8150,15.8150
0,600009.SH,2015-01-08,46,20.18,20.18,19.73,20.00,20.00,198117.45,-0.016008,20.663897,19.736103,12.828915,-90.842491,100.000000,20.2000,20.2000
0,600016.SH,2015-01-08,47,10.61,10.66,10.09,10.20,10.20,4851684.17,-0.018247,10.957604,9.997396,11.862558,-99.887006,100.000000,10.4775,10.4775
0,600028.SH,2015-01-08,48,7.09,7.41,6.83,6.85,6.85,8190902.35,-0.008227,7.342000,6.743000,27.409248,36.578171,64.934862,7.0425,7.0425
0,600030.SH,2015-01-08,49,36.40,36.70,34.68,35.25,35.25,6376268.69,0.032910,36.576444,33.808556,61.517448,47.947020,100.000000,35.1925,35.1925


In [13]:
train.shape

(16695, 17)

In [14]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension * (len(config.INDICATORS) + 2) + 1
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 15, State Space: 151


In [20]:
## 如果不加这个，会报错的。
train["date"] = train.time

In [21]:
### Train

env_kwargs = {
    "stock_dim": stock_dimension,
    "hmax": 1000,
    "initial_amount": 1000000,
    "buy_cost_pct": 6.87e-5,
    "sell_cost_pct": 1.0687e-3,
    "reward_scaling": 1e-4,
    "state_space": state_space,
    "action_space": stock_dimension,
    "tech_indicator_list": config.INDICATORS,
    "print_verbosity": 1,
    "initial_buy": True,
    "hundred_each_trade": False, ## 如果为True，会报错，可能是库代码写的问题。
}

e_train_gym = StockTradingEnv(df=train, **env_kwargs)

In [24]:
## DDPG

env_train, _ = e_train_gym.get_sb_env()
print(f"print(type(env_train)): {print(type(env_train))}")

agent = DRLAgent(env=env_train)
DDPG_PARAMS = {
    "batch_size": 256,
    "buffer_size": 50000,
    "learning_rate": 0.0005,
    "action_noise": "normal",
}
POLICY_KWARGS = dict(net_arch=dict(pi=[64, 64], qf=[400, 300]))
model_ddpg = agent.get_model(
    "ddpg", model_kwargs=DDPG_PARAMS, policy_kwargs=POLICY_KWARGS
)

trained_ddpg = agent.train_model(
    model=model_ddpg, tb_log_name="ddpg", total_timesteps=10000
)

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>
print(type(env_train)): None
{'batch_size': 256, 'buffer_size': 50000, 'learning_rate': 0.0005, 'action_noise': NormalActionNoise(mu=[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], sigma=[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1])}
Using cpu device
Logging to tensorboard_log/ddpg/ddpg_1
Episode: 2
day: 1112, episode: 2
begin_total_asset: 1000000.00
end_total_asset: 1596003.66
total_reward: 596003.66
total_cost: 11603.15
total_trades: 1363
Sharpe: 0.483
Episode: 3
day: 1112, episode: 3
begin_total_asset: 1000000.00
end_total_asset: 1511559.50
total_reward: 511559.50
total_cost: 411.90
total_trades: 73
Sharpe: 0.459
Episode: 4
day: 1112, episode: 4
begin_total_asset: 1000000.00
end_total_asset: 1511809.50
total_reward: 511809.50
total_cost: 411.87
total_trades: 74
Sharpe: 0.460
Episode: 5
day: 1112, episode: 5
begin_total_asset: 1000000.00
end_total_asset: 1514541.05
total_reward: 514541.05
total_cost

In [25]:
## A2C

agent = DRLAgent(env=env_train)
model_a2c = agent.get_model("a2c")

trained_a2c = agent.train_model(
    model=model_a2c, tb_log_name="a2c", total_timesteps=50000
)


{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device
Logging to tensorboard_log/a2c/a2c_1
----------------------------------------
| time/                 |              |
|    fps                | 915          |
|    iterations         | 100          |
|    time_elapsed       | 0            |
|    total_timesteps    | 500          |
| train/                |              |
|    entropy_loss       | -21.5        |
|    explained_variance | -9.69        |
|    learning_rate      | 0.0007       |
|    n_updates          | 99           |
|    policy_loss        | -9.16        |
|    reward             | -0.015446212 |
|    std                | 1.01         |
|    value_loss         | 0.238        |
----------------------------------------
----------------------------------------
| time/                 |              |
|    fps                | 912          |
|    iterations         | 200          |
|    time_elapsed       | 1            |
|    total_timesteps    | 1

In [28]:
### Trade

trade = p.data_split(p.dataframe, TRADE_START_DATE, TRADE_END_DATE)
trade["date"] = trade.time

env_kwargs = {
    "stock_dim": stock_dimension,
    "hmax": 1000,
    "initial_amount": 1000000,
    "buy_cost_pct": 6.87e-5,
    "sell_cost_pct": 1.0687e-3,
    "reward_scaling": 1e-4,
    "state_space": state_space,
    "action_space": stock_dimension,
    "tech_indicator_list": config.INDICATORS,
    "print_verbosity": 1,
    "initial_buy": False,
    "hundred_each_trade": True,
}
e_trade_gym = StockTradingEnv(df=trade, **env_kwargs)

df_account_value, df_actions = DRLAgent.DRL_prediction(
    model=trained_ddpg, environment=e_trade_gym
)

df_actions.to_csv("action.csv", index=False)
df_actions

Episode: 2
day: 103, episode: 2
begin_total_asset: 1000000.00
end_total_asset: 1291942.34
total_reward: 291942.34
total_cost: 68.66
total_trades: 39
Sharpe: 3.370
hit end!


,600000.SH,600009.SH,600016.SH,600028.SH,600030.SH,600031.SH,600036.SH,600050.SH,600104.SH,600196.SH,600276.SH,600309.SH,600519.SH,600547.SH,600570.SH
date,,,,,,,,,,,,,,,
2019-08-01,0,0,1000,1000,0,1000,0,0,0,0,1000,1000,0,0,0
2019-08-01,0,0,1000,1000,0,1000,0,0,0,0,1000,1000,0,0,0
2019-08-02,0,0,1000,1000,0,1000,0,0,0,0,1000,1000,0,0,0
2019-08-05,0,0,1000,1000,0,1000,0,0,0,0,1000,1000,0,0,0
2019-08-06,0,0,1000,1000,0,1000,0,0,0,0,1000,1000,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2019-12-25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2019-12-26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [35]:
ReturnPlotter?

Init signature: ReturnPlotter(df_account_value, df_trade, start_date, end_date)
Docstring:     
An easy-to-use plotting tool to plot cumulative returns over time.
Baseline supports equal weighting(default) and any stocks you want to use for comparison.
File:           ~/Desktop/FinRL-Meta-master/meta/data_processors/tushare.py
Type:           type
Subclasses:     

In [36]:
### Backtest

# matplotlib inline
df_account_value["time"] = df_account_value.date
plotter = ReturnPlotter(df_account_value, trade, TRADE_START_DATE, TRADE_END_DATE)
# plotter.plot_all()

plotter.plot()

# matplotlib inline
# # ticket: SSE 50：000016
# plotter.plot("000016")

ValueError: x and y must have same first dimension, but have shapes (103,) and (104,)

In [38]:
df_account_value.time.value_counts()

time
2019-08-01    1
2019-11-08    1
2019-11-22    1
2019-11-21    1
2019-11-20    1
             ..
2019-09-16    1
2019-09-12    1
2019-09-11    1
2019-09-10    1
2019-12-31    1
Name: count, Length: 103, dtype: int64

In [39]:


#### Use pyfolio

# CSI 300
baseline_df = plotter.get_baseline("399300")


daily_return = plotter.get_return(df_account_value)
daily_return_base = plotter.get_return(baseline_df, value_col_name="close")

perf_func = timeseries.perf_stats
perf_stats_all = perf_func(
    returns=daily_return,
    factor_returns=daily_return_base,
    positions=None,
    transactions=None,
    turnover_denom="AGB",
)
print("==============DRL Strategy Stats===========")
print(f"perf_stats_all: {perf_stats_all}")


daily_return = plotter.get_return(df_account_value)
daily_return_base = plotter.get_return(baseline_df, value_col_name="close")

perf_func = timeseries.perf_stats
perf_stats_all = perf_func(
    returns=daily_return_base,
    factor_returns=daily_return_base,
    positions=None,
    transactions=None,
    turnover_denom="AGB",
)
print("==============Baseline Strategy Stats===========")

print(f"perf_stats_all: {perf_stats_all}")

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
Remote end closed connection without response
Remote end closed connection without response
Remote end closed connection without response


OSError: 获取失败，请检查网络.